# Con la simulación, vamos a calcular los espectros de potencia, alm solos de la sobredensidad, luego aplicando la máscara del plano, luego aplicando la máscara de sagittarius, y finalmente aplicando las dos máscaras a la simulación 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
from astropy.io import ascii
from astropy.coordinates import SkyCoord, CartesianRepresentation
from astropy.wcs import WCS
import gala.coordinates as gc
import healpy as hp
from pylab import cm
import matplotlib as mpl
from matplotlib.patches import Circle, PathPatch, Ellipse
from healpy.newvisufunc import projview, newprojplot
from mpl_toolkits.mplot3d import Axes3D
from astropy.coordinates import cartesian_to_spherical
from matplotlib.lines import Line2D
from matplotlib.path import Path
from matplotlib.legend_handler import HandlerTuple
import datosCM
import pymaster as nmt

from matplotlib.patches import Rectangle

In [ ]:
from astropy.coordinates import SkyCoord

def cat_to_hpx(lon, lat, nside, radec=False):   #radec=False: si vamos de cartesianas a esfericas galacticas
    """
    Convierte un catálogo a un mapa HEALPix de densidad de número, es decir,
    el número de estrellas por grado cuadrado de cielo.

    Parámetros
    ----------
    lon, lat : (arreglo, arreglo)
        Coordenadas de las fuentes en grados.
        Si radec=True, asume que la entrada está en el sistema ICRS,
        De lo contrario, asume que la entrada está en latitud y longitud galáctica.

    nside : int
        Número HEALPix nside del mapa de destino, define el número de píxeles.

    radec : bool
        Cambio entre Ra/Dec y l/b (galáctico) como sistema de coordenadas de entrada.

    Return
    ------
    hpx_map : arreglo
        Mapa HEALPix de los recuentos de número del catálogo en coordenadas galácticas
    """
    npix = hp.nside2npix(nside)

    if radec:
        eq = SkyCoord(lon, lat, unit='deg')
        l, b = eq.galactic.l.value, eq.galactic.b.value
    else:
        l, b = lon, lat

    # OJO ACA  

    theta = np.radians(90. - b)                 # latitud en radianes
    phi = np.radians(l)               # longitud en radianes        
    indices = hp.ang2pix(nside, theta, phi, lonlat=False)  


    indx, counts = np.unique(indices, return_counts=True)

    # llenar el mapa de cielo completo
    hpx_map = np.zeros(npix, dtype=int)
    hpx_map[indx] = counts

    return hpx_map


In [ ]:
fwhm2sigma = lambda fwhm: fwhm / np.sqrt(8*np.log(2))


# Parámetros a usar 

In [ ]:
nside=128
lmax=3*nside-1
#lmax=47
espesor=5 #kpc
radio=70 #kpc

radio_min=radio-espesor/2
radio_max=radio+espesor/2

# Datos Gaia

In [ ]:
# usecols agrega solo las columnas con las que quiero trabajar
datos=pd.read_csv('GK-RR_cm_magallanic_system_final.csv',usecols=['source_id','ra','dec','d_kpc_G','l','b','dp_LMC', 'lp_LMC','bp_LMC','xp_LMC','yp_LMC','zp_LMC',
                                                                  'X_al_CG','Y_al_CG','Z_al_CG','X_al_sol','Y_al_sol','Z_al_sol','vxp_LMC','vyp_LMC', 'vzp_LMC', 'vx', 'vy', 'vz'])
#datos=pd.read_csv('/home/marz/Dropbox/PlanAB/gkrr/GK-RR_cm_magallanic_system_final.csv',usecols=['source_id','l','b','d_kpc_G','dp_LMC', 'lp_LMC','bp_LMC','xp_LMC','yp_LMC','zp_LMC',
#                                                                   'X_al_CG','Y_al_CG','Z_al_CG','X_al_sol','Y_al_sol','Z_al_sol','vxp_LMC','vyp_LMC', 'vzp_LMC', 'vx', 'vy', 'vz'])

datos

## Usando la condición del radio $R =70 $ kpc con un espesor de $\pm 5$ kpc. 

### Aquí datos['d_kpc_G']

datos_gal2= SkyCoord(ra=datos['ra']*u.deg, dec=datos['dec']*u.deg, distance=datos['d_kpc_G']*u.kpc,frame='icrs')
datos_galactocentric=datos_gal2.transform_to(frame='galactocentric')
dd=pd.DataFrame()
x=datos_galactocentric.cartesian.x.value
y=datos_galactocentric.cartesian.y.value
z=datos_galactocentric.cartesian.z.value
spherical_coords = cartesian_to_spherical(x, y, z)
r = spherical_coords[0]#.to(u.Mpc, equivalencies=u.dimensionless_angles()).value
theta = spherical_coords[1].to(u.rad).value
phi = spherical_coords[2].to(u.rad).value
phi2 = spherical_coords[2].to(u.degree).value
theta
#Chequear las coordenadas de los datos (galactico o galactocentrico)
#Usar el mismo sistema de coordenadas
#chequear el sistema de las coordenadas y las velocidades de los datos

datos_gal2= SkyCoord(ra=datos['ra']*u.deg, dec=datos['dec']*u.deg, distance=datos['d_kpc_G']*u.kpc,frame='icrs')
datos_galactocentric=datos_gal2.transform_to(frame='galactic')
dd=pd.DataFrame()
x=datos_galactocentric.cartesian.x.value
y=datos_galactocentric.cartesian.y.value
z=datos_galactocentric.cartesian.z.value
spherical_coords = cartesian_to_spherical(x, y, z)
r = spherical_coords[0]#.to(u.Mpc, equivalencies=u.dimensionless_angles()).value
theta = spherical_coords[1].to(u.rad).value
phi = spherical_coords[2].to(u.rad).value
phi2 = spherical_coords[2].to(u.degree).value
phi2,datos_gal2.galactic.l.deg,datos['l']
#Chequear las coordenadas de los datos (galactico o galactocentrico)
#Usar el mismo sistema de coordenadas

In [ ]:
#defino un skycoord con las coordenadas galacticas
datos['cotheta'] = np.pi/2 - datos['b']*np.pi/180
datos['phi'] = datos['l']*np.pi/180
#velocidades
datos['vr'] = np.sin(datos['cotheta']) * np.cos(datos['phi'])* datos['vx'] + np.sin(datos['cotheta']) * np.sin(datos['phi']) * datos['vy'] + np.cos(datos['cotheta']) * datos['vz']
datos['vtheta'] = np.cos(datos['cotheta'])* np.cos(datos['phi']) * datos['vx'] + np.sin(datos['phi']) * np.cos(datos['cotheta']) * datos['vy'] - np.sin(datos['cotheta']) * datos['vz']
datos['vphi'] =  ( np.sin(datos['phi'])) * datos['vx'] -  np.cos(datos['phi']) * datos['vy']
#condicion de radio
datos_condicion =datos[(datos['d_kpc_G']<radio_max) & (datos['d_kpc_G']>radio_min)]
datos_condicion.reset_index(drop=True, inplace=True)
print(len(datos_condicion))
#print(datos)
l = coord.Angle(datos_condicion['l'].values*u.degree)
l = l.wrap_at(180*u.degree)
b = coord.Angle(datos_condicion['b'].values*u.degree)

datos_gal= SkyCoord(l=l, b=b, frame='galactic')
ipix_datos = hp.ang2pix(nside, np.pi/2- datos_gal.galactic.b.radian, datos_gal.galactic.l.radian)
print(nside)

In [ ]:
#sobredensidad (no suavizado)
l = datos_gal.galactic.l.degree
b = datos_gal.galactic.b.degree
mapa_datos = cat_to_hpx(l,b, nside=nside, radec=False)
mapa_datos_mean = mapa_datos.mean()
mapa_datos = (mapa_datos - mapa_datos_mean) / mapa_datos_mean

# Suavizar el mapa SOLO para visualización
mapa_datos_smooth= hp.sphtfunc.smoothing(mapa_datos, sigma=np.radians(fwhm2sigma(30)))

# Crear máscara del plano galáctico |b| < 10°
xg, yg = np.meshgrid(
    np.pi/2 - np.linspace(np.radians(-10), np.radians(10), 10000),
    np.linspace(0, 2*np.pi, 10000)
)
mask = hp.ang2pix(nside, xg, yg)

# Crear copia enmascarada del mapa suavizado solo
mapa_datos_smooth_masked_plano = mapa_datos_smooth.copy()
mapa_datos_smooth_masked_plano[mask] = 0

# Crear copia enmascarada del mapa NO suavizado (para calculos  de alm y cls)
mapa_datos_masked_plano = mapa_datos.copy()
mapa_datos_masked_plano[mask] = 0

In [ ]:
len(mapa_datos_masked_plano[mapa_datos_masked_plano<-1])

# 1) sobredensidad con máscara del plano de la galaxia 

In [ ]:

# 
fig = plt.figure(figsize=(9, 5))
ax = fig.add_subplot()
mmin, mmax = np.min(mapa_datos_smooth_masked_plano[np.isfinite(mapa_datos_smooth_masked_plano)]), np.max(mapa_datos_smooth_masked_plano[np.isfinite(mapa_datos_smooth_masked_plano)])



projview(
    mapa_datos_smooth_masked_plano,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
    title='Map: Data smooth, Galactic Center '+str(radio)+' kpc',
    #cbar_ticks=[-1, 0, 1, 2],
    #cmap="turbo",
    xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-1,
    #max=mmax,
    projection_type="mollweide",
    flip='astro',
    zorder=1,
    hold=True
)

plt.tight_layout()
plt.savefig('moll_mapa_datos_masked_smooth_'+str(radio)+'.pdf')
plt.show()

print(len(mapa_datos_smooth_masked_plano))


# Calculamos los $C_{\ell}$  y $a_{\ell m}$ para la condicion de 70 kpc, 350 estrellas, con ```anafast()```

In [ ]:
cl_sobre70_datos = hp.anafast(mapa_datos_masked_plano, lmax=lmax, alm=True, pol=False)

print(cl_sobre70_datos)

## $C_{\ell}$ vs $(2 \ell + 1) C_{\ell}$ de los datos

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(8,4))
fig.suptitle('Data: Power Spectrum, '+str(radio)+' kpc')

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_sobre70_datos[0][:lmax + 1], 
         label="$C_{\ell}$ datos en radio "+str(radio), marker="o", linestyle="--", color='magenta', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1, 15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(0, 1)
ax[0].legend()

ax[1].plot(ells, (2*ells +1)*cl_sobre70_datos[0][:lmax + 1], 
         label="($2 \ell + 1) C_{\ell}$ datos en radio "+str(radio), marker="o", linestyle="--", color='cyan', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1, 15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(0, 3)
ax[1].legend()

plt.savefig('cl_sobre_datos_l15_'+str(radio)+'.pdf')  

plt.tight_layout()
plt.show()

# ojo l=0 y l=2 para ambos graficos 


## Hacemos el mapa reconstruido de los alm [1] con ```alm2map``` de los datos

In [ ]:
mapa_reconstruido_datos= hp.alm2map(cl_sobre70_datos[1], nside, lmax=lmax)

# sobredensidad

mapa_reconstruido_datos_sm = hp.sphtfunc.smoothing(mapa_reconstruido_datos, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

fig=plt.figure(figsize=(9,5))
ax = fig.add_subplot()
hp.projview(
  mapa_reconstruido_datos_sm,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
   title='Map: Overdensity Data smooth (alm2map), ('+str(radio)+') kpc',
    #cbar_ticks=[-1, 0, 1, 2],
    #cmap="turbo" ,
   xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-1,
    #max=40,
    projection_type="mollweide",flip='astro',zorder=1,hold=True)
hp.newprojplot(np.deg2rad(90-(-33)), np.deg2rad(279-360.0),  marker=r"*", color="k", markersize=20,markeredgewidth=0.01)

plt.savefig('moll-mapa_reconstruido_datos_smooth_'+str(radio)+'.pdf')
plt.plot()

### Resta del mapa de sobredensidad original y el reconstruido con  ```alm2map```, datos

In [ ]:
mapa_resta_datos= mapa_datos_masked_plano - mapa_reconstruido_datos

# sobredensidad

mapa_resta_datos_sm = hp.sphtfunc.smoothing(mapa_resta_datos, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

fig=plt.figure(figsize=(9,5))
ax = fig.add_subplot()
hp.projview(
  mapa_resta_datos,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
   title='Map: Overdensity (Original  - alm2map), ('+str(radio)+') kpc',
    #cmap="turbo" ,
   xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-40,
    #max=40,
    projection_type="mollweide",flip='astro',zorder=1,hold=True)
hp.newprojplot(np.deg2rad(90-(-33)), np.deg2rad(279-360.0),  marker=r"*", color="k", markersize=20,markeredgewidth=0.01)

plt.savefig('mapa_datos-resta_alm2map-original_'+str(radio)+'.pdf')
plt.plot()

In [ ]:
mapa_resta_datos= mapa_datos_masked_plano - mapa_reconstruido_datos


mapa_resta_datos_sm = hp.sphtfunc.smoothing(mapa_resta_datos, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

hp.mollview(mapa_resta_datos_sm, coord=["G"], title='Map Overdensity: Smooth Data (original - alm2map), ('+str(radio)+') kpc', unit=r"$\Delta \rho /\ \bar{\rho}$")

plt.savefig('mapa_datos-resta_alm2map-original_suavizado_'+str(radio)+'.pdf')
plt.show()

In [ ]:
print(len(mapa_resta_datos[np.abs(mapa_resta_datos) < 1]))
print(len(mapa_resta_datos))

In [ ]:
plt.hist(mapa_resta_datos, bins=1000)
plt.xlim(-20,20)

### Para todos los m, datos 

In [ ]:

matriz_alm_sobre70_datos=pd.DataFrame()
mm=[]
ll=[]
for m in np.arange(0,lmax+1,1,dtype=int):
    for l in np.arange(m,lmax+1,1,dtype=int):
        mm.append(m)
        ll.append(l)
matriz_alm_sobre70_datos['m']=mm
matriz_alm_sobre70_datos['l']=ll
matriz_alm_sobre70_datos['alm']=cl_sobre70_datos[1] #estos son los alm que se obtuvieron del anafast

matriz_alm_sobre70_datos

print(cl_sobre70_datos[1])
print('nside', nside)

In [ ]:


m_row = np.arange(7)  # m = 0 a 6
nrows = len(m_row)
ncols = 3  # Re(alm), Im(alm), |alm|

fig, axs = plt.subplots(nrows, ncols, figsize=(20, 4 * nrows))
fig.suptitle(r'$a_{\ell m}$ Overdensity Data: real, imaginary, and modulus componenets, $\ell = 47$, (' + str(radio) + r')', fontsize=16)
plt.subplots_adjust(hspace=0.5, wspace=0.3)

for i in range(nrows):
    mask = matriz_alm_sobre70_datos['m'] == m_row[i]
    l_vals = matriz_alm_sobre70_datos[mask]['l']
    alm_vals = matriz_alm_sobre70_datos[mask]['alm']

    # Re(alm)
    axs[i, 0].plot(l_vals, np.real(alm_vals), label="Re(alm)", marker="o", linestyle="--", color='red', alpha=0.7)
    axs[i, 0].set_xlabel('l')
    axs[i, 0].set_xlim(-1, 47)
    axs[i, 0].set_ylabel('Re(alm)')
    axs[i, 0].legend()

    # Im(alm)
    axs[i, 1].plot(l_vals, np.imag(alm_vals), label="Im(alm)", marker="o", linestyle="--", color='green', alpha=0.7)
    axs[i, 1].set_xlabel('l')
    axs[i, 1].set_xlim(-1, 47)
    axs[i, 1].set_ylabel('Im(alm)')
    axs[i, 1].legend()
    axs[i, 1].set_title(f'm={m_row[i]}, lmax={lmax}, nside={nside}', fontsize=11)

    # |alm|
    axs[i, 2].plot(l_vals, np.abs(alm_vals), label="|alm|", marker="o", linestyle="--", color='gray')
    axs[i, 2].set_xlabel('l')
    axs[i, 2].set_xlim(-1, 47)
    axs[i, 2].set_ylabel('|alm|')
    axs[i, 2].legend()

plt.savefig(f'alm_sobre70_datos-re-im-abs_m0-6_l15_{radio}.pdf')


In [ ]:

m_values = np.arange(0, 7)

plt.figure(figsize=(10, 6))


# por cada m para graficar su curva Re[alm] vs l
for m in m_values:
    df_m = matriz_alm_sobre70_datos[matriz_alm_sobre70_datos['m'] == m]
    l_vals = df_m['l'].values
    alm_real = np.real(df_m['alm'].values)
    plt.plot(l_vals, alm_real, label=f'm = {m}', marker='o')

plt.xlabel(r'$\ell$')
plt.xlim(-0.5, 15)
plt.ylabel(r'Re[$a_{\ell m}$]')
plt.title(r'Real component  $a_{\ell m}$ Overdensity Data: $m = 0$ to $6, \ell= 15$, ('+str(radio)+') kpc')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('alm_datos-re_'+str(radio)+'_m0-6l15.pdf')
#plt.savefig('alm_mask-re_'+str(nside)+'_m0-6l47.png')
plt.show()


In [ ]:
#### 


# Colores para distintos m
colores = ['b', 'm', 'g', 'orange', 'gray', 'black']

# Graficar |alm| por m
def plot_alm_from_df(matriz_alm_sobre70_datos, title):
    plt.figure(figsize=(6, 4))
    for m in range(6):  # m de 0 a 5
        subset = matriz_alm_sobre70_datos[matriz_alm_sobre70_datos['m'] == m]
        plt.scatter(subset['l'], np.abs(subset['alm']),
                    color=colores[m], label=f'm = {m}', s=40)

    plt.xlabel(r'$\ell$')
    plt.xlim(-0.5, 10.5)

    plt.ylabel(r'$|a_{\ell m}|$')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('alm_datos_'+str(radio)+'_abs-l10.pdf')
    #plt.savefig('alm_mask'+str(nside)+'_abs-l10.png')
    plt.show()

# Ejemplo: sobredensidad
plot_alm_from_df(matriz_alm_sobre70_datos, r'Overdensity Data: $|a_{\ell m}|$, ('+str(radio)+') kpc, $\ell=10$')



print(nside)


# 2) Aplicando la máscara de saggitarius 



In [ ]:
from astropy.coordinates.matrix_utilities import matrix_transpose, rotation_matrix
from astropy.coordinates import frame_transform_graph

In [ ]:
class Sagittarius(coord.BaseCoordinateFrame):
    """
    A Heliocentric spherical coordinate system defined by the orbit
    of the Sagittarius dwarf galaxy, as described in
        https://ui.adsabs.harvard.edu/abs/2003ApJ...599.1082M
    and further explained in
        https://www.stsci.edu/~dlaw/Sgr/.

    Parameters
    ----------
    representation : `~astropy.coordinates.BaseRepresentation` or None
        A representation object or None to have no data (or use the other keywords)
    Lambda : `~astropy.coordinates.Angle`, optional, must be keyword
        The longitude-like angle corresponding to Sagittarius' orbit.
    Beta : `~astropy.coordinates.Angle`, optional, must be keyword
        The latitude-like angle corresponding to Sagittarius' orbit.
    distance : `~astropy.units.Quantity`, optional, must be keyword
        The Distance for this object along the line-of-sight.
    pm_Lambda_cosBeta : `~astropy.units.Quantity`, optional, must be keyword
        The proper motion along the stream in ``Lambda`` (including the
        ``cos(Beta)`` factor) for this object (``pm_Beta`` must also be given).
    pm_Beta : `~astropy.units.Quantity`, optional, must be keyword
        The proper motion in Declination for this object (``pm_ra_cosdec`` must
        also be given).
    radial_velocity : `~astropy.units.Quantity`, optional, keyword-only
        The radial velocity of this object.

    """

    default_representation = coord.SphericalRepresentation
    default_differential = coord.SphericalCosLatDifferential

    frame_specific_representation_info = {
        coord.SphericalRepresentation: [
            coord.RepresentationMapping('lon', 'Lambda'),
            coord.RepresentationMapping('lat', 'Beta'),
            coord.RepresentationMapping('distance', 'distance')]
    }

In [ ]:
SGR_PHI = (180 + 3.75) * u.degree # Euler angles (from Law & Majewski 2010)
SGR_THETA = (90 - 13.46) * u.degree
SGR_PSI = (180 + 14.111534) * u.degree

# Generate the rotation matrix using the x-convention (see Goldstein)
SGR_MATRIX = (
    np.diag([1.,1.,-1.])
    @ rotation_matrix(SGR_PSI, "z")
    @ rotation_matrix(SGR_THETA, "x")
    @ rotation_matrix(SGR_PHI, "z")
)

In [ ]:
@frame_transform_graph.transform(coord.StaticMatrixTransform, coord.Galactic, Sagittarius)
def galactic_to_sgr():
    """ Compute the transformation matrix from Galactic spherical to
        heliocentric Sgr coordinates.
    """
    return SGR_MATRIX

In [ ]:
@frame_transform_graph.transform(coord.StaticMatrixTransform, Sagittarius, coord.Galactic)
def sgr_to_galactic():
    """ Compute the transformation matrix from heliocentric Sgr coordinates to
        spherical Galactic.
    """
    return matrix_transpose(SGR_MATRIX)

### Leo de nuevo los datos y aplico la condición

In [ ]:
datos

In [ ]:
datos_condicion =datos[(datos['d_kpc_G']<radio_max) & (datos['d_kpc_G']>radio_min)]
datos_condicion.reset_index(drop=True, inplace=True)
print(len(datos_condicion))
datos_condicion

l = datos_condicion['l'].values
b = datos_condicion['b'].values


In [ ]:
mapa_datos = cat_to_hpx(l, b, nside=nside, radec=False)
mapa_datos_mean = mapa_datos.mean()
#sobredensidad (no suavizado)
mapa_datos = (mapa_datos - mapa_datos_mean) / mapa_datos_mean

# Suavizar el mapa SOLO para visualización
mapa_datos_smooth= hp.sphtfunc.smoothing(mapa_datos, sigma=np.radians(fwhm2sigma(30)))

npix = hp.nside2npix(nside)

theta, phi = hp.pix2ang(nside, np.arange(npix))
l_pix = np.degrees(phi)
b_pix = 90 - np.degrees(theta)

# Transformamos pixeles a Sagitario
coords_pix = SkyCoord(l=l_pix*u.degree, b=b_pix*u.degree, frame='galactic')
coords_pix_sgr = coords_pix.transform_to(Sagittarius)

Beta_pix = coords_pix_sgr.Beta.deg

# mask sgr
mask_sgr = (np.abs(Beta_pix) < 15)  # 15 grados


In [ ]:
# copia enmascarada del mapa suavizado SOLO con máscara de Sagitario
mapa_datos_smooth_masked_sgr = mapa_datos_smooth.copy()
mapa_datos_smooth_masked_sgr[mask_sgr] = 0

# copia enmascarada del mapa NO suavizado (para Clℓ), SOLO con máscara de Sagitario
mapa_datos_masked_sgr = mapa_datos.copy()
mapa_datos_masked_sgr[mask_sgr] = 0



In [ ]:


# sobredensidad

mapa_datos_masked_sgr_sm = hp.sphtfunc.smoothing(mapa_datos_masked_sgr, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

fig=plt.figure(figsize=(9,5))
ax = fig.add_subplot()
hp.projview(
  mapa_datos_masked_sgr,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
   title='Map: Overdensity Data Sgr (alm2map), ('+str(radio)+') kpc',
    #cbar_ticks=[-1, 0, 1, 2],
    #cmap="turbo" ,
   xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-40,
    #max=40,
    projection_type="mollweide",flip='astro',zorder=1,hold=True)
hp.newprojplot(np.deg2rad(90-(-33)), np.deg2rad(279-360.0),  marker=r"*", color="k", markersize=20,markeredgewidth=0.01)

plt.savefig('moll-mapa_datos_sgr_'+str(radio)+'.pdf')
plt.plot()

In [ ]:

hp.mollview(mapa_datos_smooth_masked_sgr, title='Map: Overdensity Data Sgr Smooth, ('+str(radio)+') kpc')
plt.savefig('moll-mapa_datos_sgr_suavizado_'+str(radio)+'.pdf')
plt.show()


## Usando el mismo mapa de los datos pero aplicando la mascara de sgr

In [ ]:
cl_mapa_datos_sgr = hp.anafast(mapa_datos_masked_sgr, lmax=lmax, alm=True, pol=False)

print(cl_mapa_datos_sgr)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))
fig.suptitle('Map: Data Sagittarius Power Spectra, '+str(radio)+'kpc')

ells = np.arange(lmax + 1)

ax[0].plot(ells, cl_mapa_datos_sgr[0][:lmax + 1], 
         label="$C_{\ell}$ Sgr", marker="o", linestyle="--", color='magenta', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-0.5, 15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].legend()

ax[1].plot(ells, (2*ells +1)*cl_mapa_datos_sgr[0][:lmax + 1], 
         label="($2 \ell + 1) C_{\ell}$ Sgr", marker="o", linestyle="--", color='cyan', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-0.5, 15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(-0.5, 3)
ax[1].legend()

plt.savefig('cl_mapa_datos_sgr_'+str(radio)+'.pdf')  

plt.show()


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))
fig.suptitle('Map: Sagittarius Power Spectrum, $\ell = 15$, '+str(nside)+'')

ells = np.arange(lmax + 1)

ax[0].plot(ells, cl_mapa_datos_sgr[0][:lmax + 1], 
         label="$C_{\ell}$ Sgr", marker="o", linestyle="--", color='magenta', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-0.5, 15.5)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(0,1)
ax

ax[0].legend()

ax[1].plot(ells, (2*ells +1)*cl_mapa_datos_sgr[0][:lmax + 1], 
         label="($2 \ell + 1) C_{\ell}$ Sgr", marker="o", linestyle="--", color='cyan', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-0.5, 15.5)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(0, 3)
ax[1].legend()

plt.savefig('cl_mapa_datos_sgr_l15_'+str(nside)+'.pdf')  

plt.show()


In [ ]:
mapa_reconstruido_datos_sgr= hp.alm2map(cl_mapa_datos_sgr[1], nside, lmax=lmax)

# sobredensidad

mapa_reconstruido_datos_sgr_sm = hp.sphtfunc.smoothing(mapa_reconstruido_datos_sgr, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

fig=plt.figure(figsize=(9,5))
ax = fig.add_subplot()
hp.projview(
  mapa_reconstruido_datos_sgr,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
   title='Map: Overdensity Data Sgr (alm2map), ('+str(radio)+') kpc',
    #cbar_ticks=[-1, 0, 1, 2],
    #cmap="turbo" ,
   xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-40,
    #max=40,
    projection_type="mollweide",flip='astro',zorder=1,hold=True)
hp.newprojplot(np.deg2rad(90-(-33)), np.deg2rad(279-360.0),  marker=r"*", color="k", markersize=20,markeredgewidth=0.01)

plt.savefig('moll-mapa_reconstruido_datos_sgr_'+str(radio)+'.pdf')
plt.plot()

In [ ]:
mapa_resta_datos_sgr= mapa_datos_masked_sgr - mapa_reconstruido_datos_sgr

# sobredensidad

mapa_resta_datos_sgr_sm = hp.sphtfunc.smoothing(mapa_resta_datos_sgr, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

fig=plt.figure(figsize=(9,5))
ax = fig.add_subplot()
hp.projview(
  mapa_resta_datos_sgr,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
   title='Map: Overdensity Data Sgr (Original  - alm2map, ('+str(radio)+') kpc',
    #cmap="turbo" ,
   xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-40,
    #max=40,
    projection_type="mollweide",flip='astro',zorder=1,hold=True)
hp.newprojplot(np.deg2rad(90-(-33)), np.deg2rad(279-360.0),  marker=r"*", color="k", markersize=20,markeredgewidth=0.01)

plt.savefig('mapa_resta_datos_sgr_'+str(radio)+'.pdf')
plt.plot()

In [ ]:
hp.mollview(mapa_resta_datos_sgr_sm, title='Map: Data Sgr smooth (Original - alm2map), ('+str(radio)+')')
plt.savefig('moll_datos_resta_smooth_sgr'+str(radio)+'.pdf')

### Para todos los m, datos con máscara de sgr

In [ ]:

matriz_alm_sobre70_datos_sgr=pd.DataFrame()
mm=[]
ll=[]
for m in np.arange(0,lmax+1,1,dtype=int):
    for l in np.arange(m,lmax+1,1,dtype=int):
        mm.append(m)
        ll.append(l)
matriz_alm_sobre70_datos_sgr['m']=mm
matriz_alm_sobre70_datos_sgr['l']=ll
matriz_alm_sobre70_datos_sgr['alm']=cl_mapa_datos_sgr[1] #estos son los alm que se obtuvieron del anafast

matriz_alm_sobre70_datos_sgr

print(cl_mapa_datos_sgr[1])
print('nside', nside)

In [ ]:


m_row = np.arange(7)  # m = 0 a 6
nrows = len(m_row)
ncols = 3  # Re(alm), Im(alm), |alm|

fig, axs = plt.subplots(nrows, ncols, figsize=(20, 4 * nrows))
fig.suptitle(r'$a_{\ell m}$ Overdensity Data Sgr: real, imaginary, and modulus componenets, $\ell = 47$, (' + str(radio) + r')', fontsize=16)
plt.subplots_adjust(hspace=0.5, wspace=0.3)

for i in range(nrows):
    mask = matriz_alm_sobre70_datos_sgr['m'] == m_row[i]
    l_vals = matriz_alm_sobre70_datos_sgr[mask]['l']
    alm_vals = matriz_alm_sobre70_datos_sgr[mask]['alm']

    # Re(alm)
    axs[i, 0].plot(l_vals, np.real(alm_vals), label="Re(alm)", marker="o", linestyle="--", color='red', alpha=0.7)
    axs[i, 0].set_xlabel('l')
    axs[i, 0].set_xlim(-1, 47)
    axs[i, 0].set_ylabel('Re(alm)')
    axs[i, 0].legend()

    # Im(alm)
    axs[i, 1].plot(l_vals, np.imag(alm_vals), label="Im(alm)", marker="o", linestyle="--", color='green', alpha=0.7)
    axs[i, 1].set_xlabel('l')
    axs[i, 1].set_xlim(-1, 47)
    axs[i, 1].set_ylabel('Im(alm)')
    axs[i, 1].legend()
    axs[i, 1].set_title(f'm={m_row[i]}, lmax={lmax}, nside={nside}', fontsize=11)

    # |alm|
    axs[i, 2].plot(l_vals, np.abs(alm_vals), label="|alm|", marker="o", linestyle="--", color='gray')
    axs[i, 2].set_xlabel('l')
    axs[i, 2].set_xlim(-1, 47)
    axs[i, 2].set_ylabel('|alm|')
    axs[i, 2].legend()

plt.savefig(f'alm_sobre70_datos-sgr-re-im-abs_m0-6_l47_{radio}.pdf')


In [ ]:

m_values = np.arange(0, 7)

plt.figure(figsize=(10, 6))


# por cada m para graficar su curva Re[alm] vs l
for m in m_values:
    df_m = matriz_alm_sobre70_datos_sgr[matriz_alm_sobre70_datos_sgr['m'] == m]
    l_vals = df_m['l'].values
    alm_real = np.real(df_m['alm'].values)
    plt.plot(l_vals, alm_real, label=f'm = {m}', marker='o')

plt.xlabel(r'$\ell$')
plt.xlim(-0.5, 15)
plt.ylabel(r'Re[$a_{\ell m}$]')
plt.title(r'Real component  $a_{\ell m}$ Overdensity Data sgr mask: $m = 0$ to $6, \ell= 15$')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('alm_datos-sgr-re_'+str(radio)+'_m0-6l15.pdf')
#plt.savefig('alm_mask-re_'+str(nside)+'_m0-6l47.png')
plt.show()


In [ ]:
#### 


# Colores para distintos m
colores = ['b', 'm', 'g', 'orange', 'gray', 'black']

# Graficar |alm| por m
def plot_alm_from_df(matriz_alm_sobre70_datos_sgr, title):
    plt.figure(figsize=(6, 4))
    for m in range(6):  # m de 0 a 5
        subset = matriz_alm_sobre70_datos_sgr[matriz_alm_sobre70_datos_sgr['m'] == m]
        plt.scatter(subset['l'], np.abs(subset['alm']),
                    color=colores[m], label=f'm = {m}', s=40)

    plt.xlabel(r'$\ell$')
    plt.xlim(-0.5, 10.5)

    plt.ylabel(r'$|a_{\ell m}|$')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('alm_datos_sgr'+str(radio)+'_abs-l10.pdf')
    #plt.savefig('alm_mask'+str(nside)+'_abs-l10.png')
    plt.show()

# Ejemplo: sobredensidad
plot_alm_from_df(matriz_alm_sobre70_datos_sgr, r'Overdensity Data sgr mask: $|a_{\ell m}|$, ('+str(radio)+'), $\ell=10$')



print(nside)


# 3) Ahora aplicondo mascara del plano de la galaxia y saggitarius al mismo conjunto de datos, con 350 estrellas

## Recordaando la condición

In [ ]:
datos_condicion =datos[(datos['d_kpc_G']<radio_max) & (datos['d_kpc_G']>radio_min)]
datos_condicion.reset_index(drop=True, inplace=True)
print(len(datos_condicion))
datos_condicion

l = datos_condicion['l'].values
b = datos_condicion['b'].values


### Para cada máscara

In [ ]:
mapa_datos = cat_to_hpx(l, b, nside=nside, radec=False)
mapa_datos_mean = mapa_datos.mean()
#sobredensidad (no suavizado)
mapa_datos = (mapa_datos - mapa_datos_mean) / mapa_datos_mean

# Suavizar el mapa SOLO para visualización
mapa_datos_smooth= hp.sphtfunc.smoothing(mapa_datos, sigma=np.radians(fwhm2sigma(30)))

npix = hp.nside2npix(nside)

theta, phi = hp.pix2ang(nside, np.arange(npix))
l_pix = np.degrees(phi)
b_pix = 90 - np.degrees(theta)

# Transformamos pixeles a Sagitario
coords_pix = SkyCoord(l=l_pix*u.degree, b=b_pix*u.degree, frame='galactic')
coords_pix_sgr = coords_pix.transform_to(Sagittarius)

Beta_pix = coords_pix_sgr.Beta.deg

# mask sgr
mask_sgr = (np.abs(Beta_pix) < 15)  # 15 grados


#### mask plano 

xg, yg = np.meshgrid(
    np.pi/2 - np.linspace(np.radians(-10), np.radians(10), 10000),
    np.linspace(0, 2*np.pi, 10000)
)
mask = hp.ang2pix(nside, xg, yg)




In [ ]:
# copia masked del mapa suavizado 
mapa_datos_smooth_masked = mapa_datos_smooth.copy()
mapa_datos_smooth_masked[mask_sgr] = 0

# copia masked del mapa NO suavizado 
mapa_datos_masked = mapa_datos.copy()
#sgr
mapa_datos_masked[mask_sgr] = 0
#plano 
mapa_datos_masked[mask]=0 

In [ ]:
mapa_datos_smooth = hp.smoothing(mapa_datos_masked, sigma=np.radians(fwhm2sigma(30)))

hp.mollview(mapa_datos_masked,
            title='Map: Data Galactic Center Sgr, '+str(radio)+' kpc')

plt.savefig('moll_datos_masked_sgr_plano_'+str(radio)+'.pdf')
plt.show()



In [ ]:
mapa_datos_smooth = hp.smoothing(mapa_datos_masked, sigma=np.radians(fwhm2sigma(30)))

hp.mollview(mapa_datos_masked,
            title='Map: Data smooth Galactic Center Sgr, '+str(radio)+' kpc')

plt.savefig('moll_datos_smooth_masked_sgr_plano_'+str(radio)+'.pdf')
plt.show()



## Para los Cls y alm

In [ ]:
cl_mapa_sgr_pg = hp.anafast(mapa_datos_masked, lmax=lmax, alm=True, pol=False)

print(cl_mapa_sgr_pg)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,4))
fig.suptitle('Map: Sagittarius & PG, Power Spectra, $\ell = 15$, '+str(radio)+'')

ells = np.arange(lmax + 1)

ax[0].plot(ells, cl_mapa_sgr_pg[0][:lmax + 1], 
         label="$C_{\ell}$ Sgr + GP", marker="o", linestyle="--", color='magenta', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-0.5, 15.5)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(0,1)

ax[0].legend()

ax[1].plot(ells, (2*ells +1)*cl_mapa_sgr_pg[0][:lmax + 1], 
         label="($2 \ell + 1) C_{\ell}$ Sgr + GP", marker="o", linestyle="--", color='cyan', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1,15.5)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(0, 3)
ax[1].legend()

plt.savefig('cl_mapa_datos_sgr_pg_l15_'+str(radio)+'.pdf')  

plt.show()


In [ ]:
mapa_reconstruido_sgr_pg= hp.alm2map(cl_mapa_sgr_pg[1], nside, lmax=lmax)

# sobredensidad

mapa_reconstruido_sgr_pg_sm = hp.sphtfunc.smoothing(mapa_reconstruido_sgr_pg, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

fig=plt.figure(figsize=(9,5))
ax = fig.add_subplot()
hp.projview(
  mapa_reconstruido_sgr_pg,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
   title='Map: Overdensity Data Sgr + Galactic Plane (alm2map), ('+str(radio)+') kpc',
    #cbar_ticks=[-1, 0, 1, 2],
    #cmap="turbo" ,
   xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-40,
    #max=40,
    projection_type="mollweide",flip='astro',zorder=1,hold=True)
hp.newprojplot(np.deg2rad(90-(-33)), np.deg2rad(279-360.0),  marker=r"*", color="k", markersize=20,markeredgewidth=0.01)

plt.savefig('moll-mapa_reconstruido_datos_sgr_plano'+str(radio)+'.pdf')
plt.plot()

In [ ]:
mapa_resta_datos_sgr= mapa_datos_masked - mapa_reconstruido_sgr_pg

# sobredensidad

mapa_resta_sgr_pg_sm= hp.sphtfunc.smoothing(mapa_resta_datos_sgr, sigma=np.radians(fwhm2sigma(30))) # correspond to 30 deg

fig=plt.figure(figsize=(9,5))
ax = fig.add_subplot()
hp.projview(
  mapa_resta_datos_sgr,
    coord=["G"],
    fig=fig,
    graticule=True,
    graticule_labels=True,
    unit=r"$\Delta \rho /\ \bar{\rho}$",
    xlabel="longitude",
    ylabel="latitude",
    longitude_grid_spacing=45,
    cb_orientation="horizontal",
   title='Map: Overdensity Sgr + Galactic Plane (Original  - alm2map), ('+str(radio)+') kpc',
    #cmap="turbo" ,
   xtick_label_color='white',
    ytick_label_color='black',
    graticule_color='white',
    #min=-40,
    #max=40,
    projection_type="mollweide",flip='astro',zorder=1,hold=True)
hp.newprojplot(np.deg2rad(90-(-33)), np.deg2rad(279-360.0),  marker=r"*", color="k", markersize=20,markeredgewidth=0.01)

plt.savefig('moll-resta_datos_sgr_plano_'+str(radio)+'.pdf')
plt.plot()

# Leyendo el archivo que contiene los datos de la simulación con el subsampleo del notebook con la máscara del plano: simulacion-estadistica.ipynb

In [ ]:
df_cls = pd.read_csv('cl_completo_con_escala_radio70.csv')

df_cls

In [ ]:
cl_simu= df_cls['cl_simu'].values
cl_mean = df_cls['cl_mean'].values 
cl_std = df_cls['cl_std'].values
cl_mean_sample = df_cls['cl_mean_scaled'].values
cl_std_sample = df_cls['cl_std_scaled'].values
cl_simu70 = df_cls['cl_simu70'].values
cl_simu70_scaled = df_cls['cl_simu70_scaled'].values
cl_mean_weighted = (2 * np.arange(len(cl_mean)) + 1) * cl_mean
cl_std_weighted = (2 * np.arange(len(cl_std)) + 1) * cl_std

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,6))
fig.suptitle('Simulation: Power Spectra Galactic Plane, all stars', fontsize=16)

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_simu[:lmax + 1], 
         label="$C_{\ell}$ Simu", marker="o", linestyle="--", lw=2.5, color='orange', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1,15.5)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(-0.002, 0.07)
ax[0].legend()
ax[0].grid()


ax[1].plot(ells, (2*ells +1)*cl_simu[:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", lw=2,color='red', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1, 15.5)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(-0.005, 0.2)
ax[1].legend()
ax[1].grid()



plt.show()



In [ ]:
fig, ax = plt.subplots(1,2, figsize=(14,6))
fig.suptitle('Simulation: Power Spectra,  ('+str(radio)+')', fontsize=16)

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_simu70[:lmax + 1], 
         label="$C_{\ell}$ Simu 70 kpc", marker="o", linestyle="--", color='orange', alpha=0.7)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1,15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(-0.001, 0.04)
ax[0].legend()
ax[0].grid()

ax[1].plot(ells, (2*ells +1)*cl_simu70[:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", color='red', alpha=0.7)
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1,15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$ 70 kpc')
ax[1].set_ylim(-0.005, 0.14)
ax[1].legend()
ax[1].grid()
#plt.savefig('cl_sobre_simu_'+str(radio)+'.pdf') 

plt.show()



In [ ]:

fig, ax = plt.subplots(1,2, figsize=(14,6))
fig.suptitle('Simulation: Subsample Power Spectra,  ('+str(radio)+') kpc', fontsize=16)

ells = np.arange(lmax + 1)


ax[0].plot(ells, cl_mean, marker="o",  color='magenta', alpha=0.5,lw=2.5,label=r"$\langle C_{\ell} \rangle $ subsample N=100")
ax[0].fill_between(ells, cl_mean - cl_std, cl_mean + cl_std, color='grey', alpha=0.2, label=r"$\pm 1\sigma$")
#ax[0].bar(ell, cl_mean, yerr=cl_std, align='center', alpha=0.1, ecolor='black', capsize=1)
# Solo los puntos con barra de error
ax[0].errorbar(ells, cl_mean, yerr=cl_std, fmt='o-', color='orange',alpha=0.2,
             ecolor='black', capsize=3, label=r"")
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1, 15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(-0.001, 0.06)
ax[0].legend()
ax[0].grid()


cl_mean_weighted = (2 * ells + 1) * cl_mean
cl_std_weighted = (2 * ells + 1) * cl_std

ax[1].plot(ells, cl_mean_weighted,color='cyan',alpha=0.7,marker='o',label=r"$\langle C_{\ell} \rangle $ Subsample N=100")
ax[1].fill_between(ells, cl_mean_weighted - cl_std_weighted,
                        cl_mean_weighted + cl_std_weighted,
                        color='grey', alpha=0.2, lw=2.5, label=r"$\pm 1\sigma$ scaled ")
ax[1].errorbar(ells, cl_mean_weighted , yerr=cl_std_weighted , fmt='o-', color='blue', alpha=0.2,
             ecolor='black', capsize=3, label=r"by $(2\ell +1)$")
ax[1].set_xlabel('$\ell$')
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].legend()
ax[1].set_xlim(-1, 15)
ax[1].set_ylim(-0.008, 0.3)
ax[1].grid()


In [ ]:

fig, ax = plt.subplots(1,2, figsize=(16,8))
fig.suptitle('Simulation: Subsample Power Spectra,  ('+str(radio)+') kpc')

ells = np.arange(lmax + 1)

ax[0].plot(ells, cl_mean, marker="o",  color='magenta', alpha=0.6,lw=2.5,label=r"$\langle C_{\ell} \rangle $ subsample N=100")
ax[0].fill_between(ells, cl_mean - cl_std, cl_mean + cl_std, color='grey', alpha=0.2, label=r"$\pm 1\sigma$")
#ax[0].bar(ell, cl_mean, yerr=cl_std, align='center', alpha=0.1, ecolor='black', capsize=1)
# Solo los puntos con barra de error
ax[0].errorbar(ells, cl_mean, yerr=cl_std, fmt='o-', color='orange',alpha=0.2,
             ecolor='black', capsize=3, label=r"")


ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(-1, 15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].set_ylim(-0.001, 0.06)
ax[0].legend()
ax[0].grid()


cl_mean_weighted = (2 * ells + 1) * cl_mean
ax[1].plot(ells, (2*ells +1)*cl_mean,
          marker="o", linestyle="--", color='cyan', alpha=0.6, lw=2.5,label=r"$ (2 \ell +1) \langle C_{\ell} \rangle$ Subsample N=100")
ax[1].fill_between(ells, (2*ells +1)*cl_mean - cl_std, (2*ells +1)*cl_mean + cl_std, color='grey', alpha=0.2, label=r"$\pm 1 \sigma$ not scaled ")
ax[1].errorbar(ells, cl_mean , yerr=cl_std , fmt='o-', color='blue', alpha=0.2,
             ecolor='black', capsize=3, label=r"by $(2\ell +1)$")
ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(-1, 15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(-0.008, 0.3)
ax[1].legend()
ax[1].grid()



plt.show()



# 5) Comparando los especrtos de potencia de los datos con máscara del plano, sgr, ambas máscarcas y la simulación con solo la mascara del plano para 2150 estrellas y N=100 para la seed, a 70 kpc, y completa

In [ ]:
fig, ax = plt.subplots(6,2, figsize=(18,17))
fig.suptitle('Simulation-Data: Overdensity GC, Sgr, GC & Sgr, Simu subample,  ('+str(radio)+')')

ells = np.arange(lmax + 1)

#datos mascara plano 70 kpc
ax[0,0].plot(ells, cl_sobre70_datos[0][:lmax + 1],
         label="$C_{\ell}$ Data mask galactic center", marker="o", linestyle="--", color='orange', alpha=0.7)
ax[0,0].set_xlabel('$\ell$')
ax[0,0].set_xlim(0, 10)
ax[0,0].set_ylabel('$C_{\ell}$')
ax[0,0].set_ylim(0, 1)
ax[0,0].legend()
#datos mascra plano 70 kpc escalada por (2l+1)
ax[0,1].plot(ells, (2*ells +1)*cl_sobre70_datos[0][:lmax + 1],
         label="$(2 \ell +1) C_{\ell}$ Data mask galactic center ", marker="o", linestyle="--", color='red', alpha=0.7)
ax[0,1].set_xlabel('$\ell$')
ax[0,1].set_xlim(0, 10)
ax[0,1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[0,1].set_ylim(0, 3)
ax[0,1].legend()
#datos mascara sgr 70 kpc
ax[1,0].plot(ells, cl_mapa_datos_sgr[0][:lmax + 1],
         label="$C_{\ell}$ Sgr", marker="o", linestyle="--", color='magenta', alpha=0.7)
ax[1,0].set_xlabel('$\ell$')
ax[1,0].set_xlim(-0.5, 10)
ax[1,0].set_ylabel('$C_{\ell}$')
ax[1,0].set_ylim(0, 1)
ax[1,0].legend()
#datos mascara sgr 70 kpc escalada por (2l+1)
ax[1,1].plot(ells, (2*ells +1)*cl_mapa_datos_sgr[0][:lmax + 1],
         label="($2 \ell + 1) C_{\ell}$ Sgr", marker="o", linestyle="--", color='cyan', alpha=0.7)
ax[1,1].set_xlabel('$\ell$')
ax[1,1].set_xlim(-0.5, 10)
ax[1,1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1,1].set_ylim(-0.5, 3)
ax[1,1].legend()
#datos mascara sgr y gc 70 kpc
ax[2,0].plot(ells, cl_mapa_sgr_pg[0][:lmax + 1],
         label="$C_{\ell}$ Sgr & GC", marker="o", linestyle="--", color='blue', alpha=0.7)
ax[2,0].set_xlabel('$\ell$')
ax[2,0].set_xlim(-0.5, 10)
ax[2,0].set_ylabel('$C_{\ell}$')
ax[2,0].set_ylim(0, 1)
ax[2,0].legend()
#datos mascara sgr y gc 70 kpc escalada por (2l+1)
ax[2,1].plot(ells, (2*ells +1)*cl_mapa_sgr_pg[0][:lmax + 1],
         label="$(2 \ell + 1) C_{\ell}$ Sgr & GC", marker="o", linestyle="--", color='purple', alpha=0.7)
ax[2,1].set_xlabel('$\ell$')
ax[2,1].set_xlim(-0.5, 10)
ax[2,1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[2,1].set_ylim(0, 3)
ax[2,1].legend()
#simu 70 mascara plano kpc 
ax[3,0].plot(ells, cl_simu70[:lmax + 1],
         label="$C_{\ell}$ Simu", marker="o", linestyle="--", color='black', alpha=0.7)
ax[3,0].set_xlabel('$\ell$')
ax[3,0].set_xlim(0, 10)
ax[3,0].set_ylabel('$C_{\ell}$')
ax[3,0].set_ylim(-0.1, 1)
ax[3,0].legend()
#simu 70 mascara plano kpc escalada por (2l+1)
ax[3,1].plot(ells, (2*ells +1)*cl_simu70[:lmax + 1],
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", color='grey')
ax[3,1].set_xlabel('$\ell$')
ax[3,1].set_xlim(0, 10)
ax[3,1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[3,1].set_ylim(-0.2, 3)
ax[3,1].legend()
#simu subsample mean 70 kpc mascara del plano 
ax[4,0].plot(ells, cl_mean, marker="o",  color='magenta', alpha=0.6,lw=2.5,label=r"$\langle C_{\ell} \rangle $ subsample N=100")
ax[4,0].fill_between(ells, cl_mean - cl_std, cl_mean + cl_std, color='grey', alpha=0.2, label=r"$\pm 1\sigma$")
ax[4,0].errorbar(ells, cl_mean, yerr=cl_std, fmt='o-', color='orange',alpha=0.2,
             ecolor='black', capsize=3, label=r"")
ax[4,0].set_xlabel('$\ell$')
ax[4,0].set_xlim(0, 10)
ax[4,0].set_ylabel('$C_{\ell}$')
ax[4,0].set_ylim(-0.03, 0.06)
ax[4,0].legend()
#simu subsample mean escalado 70 kpc mascara del plano 
ax[4,1].plot(ells, cl_mean_weighted,color='cyan',alpha=0.7,marker='o',label=r"$\langle C_{\ell} \rangle $ Subsample N=100")
ax[4,1].fill_between(ells, cl_mean_weighted - cl_std_weighted,
                        cl_mean_weighted + cl_std_weighted,
                        color='grey', alpha=0.1, lw=2.5, label=r"$\pm 1\sigma$ scaled ")
ax[4,1].errorbar(ells, cl_mean_weighted , yerr=cl_std_weighted , fmt='o-', color='blue', alpha=0.2,
             ecolor='black', capsize=3, label=r"by $(2\ell +1)$")
ax[4,1].set_xlabel('$\ell$')
ax[4,1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[4,1].legend()
ax[4,1].set_xlim(-1, 15)
ax[4,1].set_ylim(-0.008, 0.3)
#ax[4,1].grid()
#toda la simu con mascara plano 
ax[5,0].plot(ells, cl_simu[:lmax + 1], 
         label="$C_{\ell}$ Simu", marker="o", linestyle="--", lw=2.5, color='orange', alpha=0.7)
ax[5,0].set_xlabel('$\ell$')
ax[5,0].set_xlim(-1,10.5)
ax[5,0].set_ylabel('$C_{\ell}$')
ax[5,0].set_ylim(-0.002, 0.07)
ax[5,0].legend()
#simu toda escalada por 2*ells +1 1 mascara plano 
ax[5,1].plot(ells, (2*ells +1)*cl_simu[:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", lw=2,color='red', alpha=0.7)
ax[5,1].set_xlabel('$\ell$')
ax[5,1].set_xlim(-1, 10.5)
ax[5,1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[5,1].set_ylim(-0.005, 0.2)
ax[5,1].legend()


plt.savefig('cl_sobre_simu-sample_vs_sgr_pg_'+str(radio)+'.pdf')
plt.show()
#plt.savefig('alm_mask-re_'+str(nside)+'_m0-6l47.png')
plt.show()

# Espectros de potencia de la simu y datos, con GC, Sgr, GC & Sgr, subsample para $C_{\ell}$ vs $(2 \ell + 1)C_{\ell}$

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(18,8))
fig.suptitle('Simulation-Data: Overdensity Power Spectra masks: GC, Sgr, GC + Sgr, Simu, Subample,  ('+str(radio)+')')

ells = np.arange(lmax + 1)

#cl para 70 kpc y toda la simulacion
ax[0].plot(ells, cl_sobre70_datos[0][:lmax + 1],
         label="$C_{\ell}$ Data GC", linestyle='--',marker="o",color='orange', alpha=0.7)
ax[0].plot(ells, cl_mapa_datos_sgr[0][:lmax + 1],
         label="$C_{\ell}$ Sgr", linestyle='--',marker="o", color='magenta', alpha=0.7)
ax[0].plot(ells, cl_mapa_sgr_pg[0][:lmax + 1],
        label="$C_{\ell}$ Sgr & GC", marker="o", linestyle="--", color='blue', alpha=0.7)
ax[0].plot(ells, cl_simu70[:lmax + 1],
         label="$C_{\ell}$ Simu 70 kpc", marker="o", linestyle="--", color='black', alpha=0.7)
#ax[0].plot(ells, cl_sobre70_simu[:lmax + 1],
 #          label="$C_{\ell}$ Simu Subsample", marker="o", linestyle="--", color='grey', alpha=0.7)
ax[0].plot(ells, cl_mean, marker="o",  color='cyan', linestyle="--", label=r"$\langle C_{\ell} \rangle $ subsample N=100")
ax[0].plot(ells, cl_simu[:lmax + 1], 
         label="$C_{\ell}$ Simu all stars", marker="o", linestyle="--", lw=2.5, color='green', alpha=0.7)

#ax[0].plot(ells, cl_sobre70_simu[0][:lmax + 1],
     #    label="$C_{\ell}$ Simu", marker="o", linestyle="--", color='green', alpha=0.7)
ax[0].set_ylim(-0.1, 1)
ax[0].set_xlabel('$\ell$')
ax[0].set_xlim(0, 15)
ax[0].set_ylabel('$C_{\ell}$')
ax[0].legend()

#mismoscl pero escalados por (2l+1) en 70 kpc y toda la simu 
ax[1].plot(ells, (2*ells +1)*cl_sobre70_datos[0][:lmax + 1],
         label="$2 \ell + 1)C_{\ell}$ Data GC", color='orange',lw=2.2,)
ax[1].plot(ells, (2*ells  +1)*cl_mapa_datos_sgr[0][:lmax + 1],
         label="($2 \ell + 1) C_{\ell}$ Sgr", color='magenta',)
ax[1].plot(ells, (2*ells +1)*cl_mapa_sgr_pg[0][:lmax + 1],
         label="$(2 \ell + 1) C_{\ell}$ Sgr & GC", color='blue',)
ax[1].plot(ells, (2*ells +1)*cl_simu70[:lmax + 1],
         label="$(2 \ell +1) C_{\ell}$ Simu 70 kpc ",  color='black')
#ax[1].plot(ells, (2*ells +1)*cl_sobre70_simu[0][:lmax + 1],
 #        label="$(2 \ell +1) C_{\ell}$ Simu ", marker="o", linestyle="--", lw=2.3,color='pink',)
ax[1].plot(ells, cl_mean_weighted,color='cyan',label=r"$\langle C_{\ell} \rangle $ Subsample N=100")
ax[1].plot(ells, (2*ells +1)*cl_simu[:lmax + 1], 
         label="$(2 \ell +1) C_{\ell}$ Simu all stars ", lw=2,color='green',)

ax[1].set_xlabel('$\ell$')
ax[1].set_xlim(0, 15)
ax[1].set_ylabel('$(2 \ell +1) C_{\ell}$')
ax[1].set_ylim(-0.1,3)
ax[1].legend()

#plt.savefig('cl_sobre_simu-sample_vs_sgr_pg_2l+1_'+str(radio)+'.pdf')
plt.show()